In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# for repreoducibility
torch.manual_seed(1)

### Softmax

In [3]:
z = torch.FloatTensor([1, 2, 3]) # |z| = (3,)
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
hypothesis.sum()

tensor(1.)

### Cross Entropy

$H(P,Q) = -\mathbb{E}_{\text{x}\sim P(\text{x})}[\log Q(\text{x})] = -\sum_{x\in\chi}P(x)\log Q(x)$

### Cross Entropy Loss
$L=\frac{1}{N}\sum-y\log(\hat{y})$, 

where $y$ is $P(x)$ and $\hat{y}$ is $Q(x)$ or $P_\theta(x)$

In [5]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


- torch.randint()

Returns a tensor filled with random integers generated uniformly between low (inclusive) and high (exclusive).

In [8]:
y_one_hot = torch.zeros_like(hypothesis) # |y_one_hot| = (3, 5)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # |y| = (3,) --> (3, 1)
# [0, 2, 1] --> 

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

- .scatter(dim, index, src)

For a 3-D tensor, self is updated as:

In [ ]:
self[index[i][j][k]][j][k] = src[i][j][k]  # if dim == 0
self[i][index[i][j][k]][k] = src[i][j][k]  # if dim == 1
self[i][j][index[i][j][k]] = src[i][j][k]  # if dim == 2

In [9]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [10]:
# more simply
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [11]:
# most simple
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

### example

In [13]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

In [18]:
# low-level
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=0.1)

n_epochs = 1000
for epoch in range(n_epochs + 1):
    
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    optimizer.zero_grad() # 이거 안하면, 계속 누적(더하기)함
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [17]:
# using F.cross_entropy
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(3, requires_grad=True)
optimizer = optim.SGD([W, b], lr=0.1)

n_epochs = 1000
for epoch in range(n_epochs + 1):
    
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    optimizer.zero_grad() # 이거 안하면, 계속 누적(더하기)함
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.623000
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285073
Epoch 1000/1000 Cost: 0.248155


In [22]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3, bi)
        
    def forward(self, x):
        return self.linear(x)

In [23]:
model = SoftmaxClassifierModel()

In [25]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

n_epochs = 1000
for epoch in range(n_epochs + 1):
    
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad() # 이거 안하면, 계속 누적(더하기)함
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, cost.item()))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
